# Задание 1

**Подключение к VK. В Jupyter Notebook нужно отразить подключение и вывод информации о пользователе на экран**

1. Работу ведем в Jupyter Notebook.

2. Изучить API VK https://dev.vk.com/ru/reference

3. Получить токен (ключ доступа) https://dev.vk.com/ru/api/access-token/authcode-flow-
user

4. Попробовать скачать json с информацией о себе (другом пользователе). 
Поля выбрать по своему усмотрению. Список полей https://dev.vk.com/ru/reference/objects/user. Вывести на экран

Токен можно получить здесь: https://vkhost.github.io/

In [ ]:
# Устанавливаем необходимые библиотеки

# !pip install vk_requests

In [ ]:
import vk_requests    # работа с VK API

## Подключение к VK API

In [ ]:
# Здесь необходимо вставить Ваш токен, передавать его никому нельзя!

token = 'vk1.a.R6HkBn5TiKdyYV7bO5xKs10EYQw8H3n_SQtp_g9qbGX9Jkzyvgz6dx6HpHfyMOVpsQR3Tbhu7NFl8N-IWCDWlNuYwSafXgn6dujWVudLGpWxTiyNaXlozPBOUTunX-Ufbxrr3XLqWQHLtikfPj3m4k2ucIKmnmaAmianP6-f2irTBXvLFO-DafW3eyC35nrBhLYm69y02aTF3sCblJPDBw'

In [ ]:
api = vk_requests.create_api(service_token=token)

## Получение данных по пользователю

In [ ]:
# Здесь необходимо вставить Ваш ID профиля

profile_id = 1

In [ ]:
api.users.get(user_id=profile_id, fields=[
    'bdate', # День рождения
    'sex', # Пол
    'personal', # Персональная информация, взгляды, языки
    'city', # Родной город
    'religion', # Религия
    'education', # Образование 
    'schools' # Школа
])

# Задание 2

**Граф друзей + DataLens + API.Геокодер**

В Jupyter Notebook нужно отразить все шаги + вывод (полученный граф друзей). Ссылку на DataLens разместить в Jupyter Notebook. Не забудьте предоставить доступ к дашборду (полный).
1. Выбрать пользователя (можно не себя).
2. Построить граф друзей (можно использовать другие способы, не тот, что был
показан).
3. Найти кластеры. Предположить, что это за кластеры друзей (работа/учеба/хобби). 
4. Изучить работу с DataLens https://cloud.yandex.ru/docs/datalens/quickstart
5. Изучить работу с API.Геокодер https://yandex.ru/dev/maps/geocoder/#addiwon + код
с занятия
6. Построить карту друзей по городам (количество друзей в каждом городе) в
DataLens.

In [ ]:
# Устанавливаем необходимые библиотеки

# !conda install --channel conda-forge pygraphviz

In [ ]:
# Выбираем пользователя

user_id = 11

## Получаем список друзей

In [ ]:
import requests
import time
import collections
from tqdm import tqdm


def get_friends_ids(api, user_id):
    json_response = api.friends.get(user_id=user_id)
    return json_response

# получаем список друзей друзей выбранного пользователя

graph = {}
friend_ids = get_friends_ids(api, user_id)

print(f'Начинаем поиск друзей и друзей друзей пользователя {user_id}')
for friend_id in tqdm(friend_ids['items'][:100]):
    try:
        graph[friend_id] = get_friends_ids(api, int(friend_id))['items']
    except:
        # случай, если пользователь удален или скрыт
        pass

## Строим граф

In [ ]:
import networkx
import pylab as plt
from tqdm import tqdm
from networkx.drawing.nx_agraph import graphviz_layout

# меняем размер окна, в котором выводится граф
plt.figure(figsize=(15, 15))

g = networkx.Graph(directed=False)

# добавляем центр графа - выбранного пользователя
g.add_node(user_id)

# для пользователя 143 мы бы получили граф с 27722 вершинами и 30467 связями
# из-за этого ограничим каждый уровень вложенности

for parent in tqdm(list(graph.keys())[:10]):
    
    # добавляем друга выбранного пользователя
    g.add_node(parent)
    
    # добавляем связь между выбранными пользователем и его другом
    g.add_edge(user_id, parent)
    
    # итерируемся по всем друзьями друга пользователя
    # добавляем их как ноды
    # и добавляем связи между другом пользователя и его друзьями
    for child in graph[parent][:10]:
        g.add_node(child)
        g.add_edge(parent, child)

print(g)        

# выводим граф с уровнем вложенности 2 
networkx.draw_networkx(g, 
                       pos=graphviz_layout(g), 
                       node_size=100, 
                       cmap=plt.cm.Blues,
                       node_color=range(len(g)))
plt.show()

## Кластеризируем граф алгоритмом Гирван — Ньюмена

In [ ]:
import itertools
from networkx.algorithms import community

# Сколько кортежей сообществ будем рассматривать
k = 1
comp = community.girvan_newman(g) 

print('Найденные кластеры')
for communities in itertools.islice(comp, k):
    [print(sorted(c), '\n') for c in communities]

## Построим карту друзей по городам

Можно воспользоваться API GeoCoder, однако доступ к нему с недавнего времени платный
Поэтому воспользуемся этой библиотекой https://pypi.org/project/geopy/1.9.1/

In [ ]:
# Устанавливаем необходимые библиотеки

# !pip install geopy

In [ ]:
import pandas as pd
import geopy
from geopy.geocoders import Nominatim

# сохраним данные в виде списка, потом переведем его в pandas dataframe
data = []

# создаем объект геолокатора, возвращающего координаты
# если выводит ошибку, поменяйте название агента
geopy.geocoders.options.default_user_agent = "dz1"
geolocator = Nominatim()

# добавляем город выбранного пользователя
user_id_town = api.users.get(user_id=user_id, fields=['city'])[0]['city']['title']
user_id_geolocation = geolocator.geocode(user_id_town)
data.append((user_id, user_id_geolocation.latitude, user_id_geolocation.longitude))

# итерируемся по всем друзьям
for parent in tqdm(list(graph.keys())[:10]):
    
    # итерируемся по всем друзьям пользователя
    try:
        parent_town = api.users.get(user_id=parent, fields=['city'])[0]['city']['title']
        parent_geolocation = geolocator.geocode(parent_town)
        data.append((parent, parent_geolocation.latitude, parent_geolocation.longitude))
    except:
        print(f'У пользователя {parent} не указан город')
    
    # итерируемся по всем друзьями друга пользователя
    for child in graph[parent][:10]:
        try:
            child_town = api.users.get(user_id=child, fields=['city'])[0]['city']['title']
            child_geolocation = geolocator.geocode(child_town)
            data.append((child, child_geolocation.latitude, child_geolocation.longitude))
        except:
            print(f'У пользователя {child} не указан город')

In [ ]:
pd_data = pd.DataFrame(data)

In [ ]:
pd_data = pd_data.drop_duplicates()

In [ ]:
# Очищенный от дубликатов датасет
pd_data

In [ ]:
pd_data.to_csv('data.csv', header=False)

## Построенная карта в Datalens

https://datalens.yandex/udhj2p10zxnij